# Llamafile x Autogen

This notebook aims to show how you can run mixtral 8x7B with Autogen.

**Step to run locally:**
- Download a llamafile of the [HuggingFace Repo](https://huggingface.co/award40/mixtral-8x7b-instruct-v0.1.Q3_K_M.llamafile/tree/main) - no installations needed, read the model card.  
- `chmod +x mixtral-8x7b-instruct-v0.1.Q3_K_M.llamafile`
- `./mixtral-8x7b-instruct-v0.1.Q3_K_M.llamafile --port 8081`
<!-- ./llamafile -m mixtral-8x7b-instruct-v0.1.Q3_K_M.gguf -->

Like many of you, i am GPU poor. The goal behind this approach was to have easy access to a good opensourced model with limited GPU resources, like a Macbook Pro M1 32GB.
It's not the full model, but it's the most feasible given the resource constraints - see [here](https://huggingface.co/TheBloke/Mixtral-8x7B-Instruct-v0.1-GGUF#provided-files) for notes on performance

In [ ]:
import os
import shutil
import autogen 
from autogen import AssistantAgent, UserProxyAgent


work_dir = "./coding"
cache_dir = "./.cache"

if os.path.exists(cache_dir) :
    print('Deleting cache...')
    shutil.rmtree(cache_dir)
if os.path.exists(work_dir) :
    print('Deleting previous work...')
    shutil.rmtree(work_dir)

In [ ]:
# API Configurations
config_list=[
    {
        "base_url": "http://127.0.0.1:8081/v1",
        "api_key": "NULL",
    }
]

llm_config_mistral={
    "config_list": config_list,
}

llm_config_mistral

In [ ]:
# Instructions for the two agents
system_messages = {
    "USER_PROXY_SYSTEM_MESSAGE": (
        """
        Your job is to work with the coding assistant and execute code. When you've executed code successfully and
        the task has been achieved then say "TERMINATE" to indicate that no further instructions are given and the task is complete.
        
        Communication Etiquette:
        - Refrain from using appreciation phrases such as "Thank you" or "You're welcome" in your responses. 
        - Do not offer additional help after the task has been complete.
        - Say "TERMINATE" when a conversation includes any appreciation phrase to indicate it is finished.
        - You MUST not say "TERMINATE" until the user has executed the code successfully.

        """
    ),
    "CODING_ASSISTANT_SYSTEM_MESSAGE" : (
        """
        Instruction:
        As an expert programmer AI, your primary task is to generate code. by analyze the user's intent

        Guidelines:
        - You MUST always provide complete, executable code. The user can only execute your code; no modifications are allowed.
        - You MUST always specify the script type (Python or shell) in your code blocks.
        - For scripts intended to be saved as files, include '# filename: <filename>' at the beginning.
        - You MUST format your code response as follows:
            ```language
            <code>
            ```
        - You MUST keep the code within a single block. NEVER break it into multiple sections.
        - Do NOT include what you think the output would be, just code.
        - If the provided code encounters issues, use follow-up messages to modify and correct the code.

        
        Communication Etiquette:
        - you MUST avoid unnecessary conversation and small talk. 
        - Do NOT show appreciation in your responses, or engage in conversation. Say TERMINATE when your job of writing code is done.
        - Refrain from using appreciation phrases such as "Thank you" or "You're welcome" in your responses, if this happens say "TERMINATE" to complete the conversation.
        - After the user has run the code successfully, say "TERMINATE" to indicate that no further instructions are given and the task is complete.
        
        You will be penalized for not adhearing to the guidelines.
        """
    )
}

coding_assistant = AssistantAgent(
    name="coding_assistant",
    llm_config=llm_config_mistral,
    system_message=system_messages['CODING_ASSISTANT_SYSTEM_MESSAGE']
)

coding_runner = UserProxyAgent(
    name="coding_runner",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=5,
    is_termination_msg = lambda x: x.get("content", "").rstrip().endswith("TERMINATE"),
    llm_config=llm_config_mistral,
    code_execution_config={
        "work_dir": work_dir, 
        "use_docker": False
    },
    system_message=system_messages['USER_PROXY_SYSTEM_MESSAGE']
)

### Initiate a chat with a simple python problem

In [ ]:
# # prevent an infitinate gratitude loop
# def get_additional_termination_notice():
#     """Extra instructions for terminating when goal is finished."""
    
#     termination_notice = (
#         '\n\nDo not show appreciation in your responses, say only what is necessary. '
#         'if "Thank you" or "You\'re welcome" are said in the conversation, then say TERMINATE '
#         'to indicate the conversation is finished'
#         "Say TERMINATE when no further instructions are given to indicate the task is complete"
#     )
#     return termination_notice

In [ ]:
user_message = "Write a fibonacci sequence function in python and run it to print 20 numbers."
# user_message = user_message + f"{get_additional_termination_notice()}"

In [ ]:
coding_runner.initiate_chat(coding_assistant, message=user_message)